In [1]:
from importlib import reload
from src import parsing
from src import model_path
from src import models_path
from src import model_goal
from src import models_goal
from src import model_interface
from src import visualization
from src import util
from src import config
reload(parsing)
reload(model_path)
reload(models_path)
reload(model_goal)
reload(models_goal)
reload(model_interface)
reload(visualization)
reload(util)
reload(config)

<module 'src.config' from '/home/deus/Documents/trajectory-prediction/src/config.py'>

In [2]:
# load input
# since we use a different file then for training we can use everything for eval
file_path = "data/medium_ab.csv"
_, eval_data, _ = parsing.parse_atc_day(file_path, train_ratio=0.02, eval_ratio=0.98)
input_path, correct_goals, ground_truth_path = eval_data

# limit amount of data used to avoid memory overflow
LIMIT = 5000
input_path = input_path[:LIMIT,:,:]
correct_goals = correct_goals[:LIMIT,:]
ground_truth_path = ground_truth_path[:LIMIT,:,:]
eval_data = input_path, correct_goals, ground_truth_path

100%|██████████| 1058/1058 [00:16<00:00, 63.97it/s] 


Dividing 580 human trajectories: 11 training, 569 eval, 0 test
train (6670, 40, 2) float64 (6670, 25) float32 (6670, 40, 2) float64
eval (240985, 40, 2) float64 (240985, 25) float32 (240985, 40, 2) float64


In [3]:
# generate estimates
m = model_goal.ModelGoal()
m.load('goal_simple_cnn')
goals_estimation = m.estimate(eval_data)

# free memory
m = None

In [8]:
reload(model_path)

# load predictive model
m = model_path.ModelPath(uses_goal=True)
m.load('path_simple_lstm_goal')

In [6]:
# evaluate gt goals
m.metrics(input_path, ground_truth=ground_truth_path, goals=correct_goals)

MED: 0.030914461425584034
FDE: 0.04430704503859379


In [9]:
# evaluate estimated goals
m.metrics(input_path, ground_truth=ground_truth_path, goals=goals_estimation)

MED: 0.035027690295997796
FDE: 0.05504289714568437


In [8]:
reload(model_path)
reload(util)
m = None
m = model_path.ModelPath(uses_goal=False)
m.load('path_simple_lstm')

# evaluate without goals
m.metrics_geometric(input_path, ground_truth=ground_truth_path)

Length ADE distances 200000, ground truth shape (5000, 40, 2)
Length FDE distances 5000, ground truth shape (5000, 40, 2)
ADE: 0.04046657588548339
